In [14]:
import requests 

In [15]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder ## to convert categorical data to numerical
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import mean_squared_error
from datetime import datetime, timedelta
import pytz


In [16]:
API_KEY = '4cef118c7f7d96c246bcd5db70793d03'
BASE_URL = 'https://api.openweathermap.org/data/2.5/'

In [17]:
def get_current_weather(city):
    url = f"{BASE_URL }weather?q={city}&{API_KEY}&units=metric"
    response = requests.get(url)
    data = response.json()
    return {
        'city' : data['name'],
        'current_temp' : round(data['main']['temp']),
        'feels_like': round(data['main']['feels_like']),
        'temp_min': round(data['main']['temp_min']),
        'temp_max': round(data['main']['temp_max']),
        'humidity': round(data['main']['humidity']),
        'description': data['weather'][0]['description'],
        'country' : data['sys']['country'],
        'Wind_Gust_Dir' : data['wind']['deg'],
        'pressure' : data['main']['pressure'],
        'Wind_gust_Speed' : data['wind']['speed'],
        
        
    }

In [18]:
def read_historical_data(filename):
    df = pd.read_csv(filename)
    df = df.dropna()
    df = df.drop_duplicates()
    return df


In [19]:
def prepare_data(data):
    le = LabelEncoder()
    data['WindGustDir'] = le.fit_transform(data['WindGustDir'])
    data['RainTomorrow'] = le.fit_transform(data['RainTomorrow'])
    
    
    X = data[['MinTemp', 'MaxTemp',  'WindGustDir', 'WindGustSpeed','Humidity', 'Pressure','Temp']]
    y = data['RainTomorrow']
    
    
    return X, y, le

In [20]:
def train_model(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2, random_state=42)
    model = RandomForestClassifier(n_estimators = 100, random_state= 42)
    model.fit(X_train, y_train)
    
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    
    print("MSE is ", mse)
    
    return model

In [21]:
def prepare_regression_data(data, feature):
    x, y = [], []
    for i in range(len(data)-1):
        x.append(data[feature].iloc[i])
        y.append(data[feature].iloc[i+1])
    
    x = np.array(x).reshape(-1,1)
    y = np.array(y)
    return x, y


In [22]:
def train_regression_model(x, y):
    x_train, x_test, y_train, y_test = train_test_split(x,y, test_size = 0.2, random_state=42)
    model = RandomForestRegressor(n_estimators = 100, random_state= 42)
    model.fit(x_train, y_train)
    
    y_pred = model.predict(x_test)
    mse = mean_squared_error(y_test, y_pred)
    
    print("MSE is ", mse)
    
    return model

In [23]:
def predict_future(model,current_value):
    predictions = [current_value]
    
    for i in range(5):
        next_value = model.predict(np.array(predictions[-1]))
        predictions.append(next_value[0])
        
    return predictions[1:]  # Exclude the initial current value

In [25]:
def weather_view():
    city = input("Enter city name: ")
    current_weather = get_current_weather(city)
    
    
    historical_data = read_historical_data('weather.csv')
    
    x, y, le = prepare_data(historical_data)
    rain_model = train_model(x, y)
    
    
    wind_deg = current_weather['Wind_Gust_Dir'] %360
    compass_points = [
    ("N", 0, 11.25), ("NNE", 11.25, 33.75), ("NE", 33.75, 56.25),
    ("ENE", 56.25, 78.75), ("E", 78.75, 101.25), ("ESE", 101.25, 123.75),
    ("SE", 123.75, 146.25), ("SSE", 146.25, 168.75), ("S", 168.75, 191.25),
    ("SSW", 191.25, 213.75), ("SW", 213.75, 236.25), ("WSW", 236.25, 258.75),
    ("W", 258.75, 281.25), ("WNW", 281.25, 303.75), ("NW", 303.75, 326.25),
    ("NNW", 326.25, 348.75)
]
    compass_direction = next(point for point, start, end in compass_points if start <= wind_deg < end)
    compass_direction_encoded = le.transform([compass_direction])[0] if compass_direction in le.classes_ else -1
    
    
    current_data = {
        'MinTemp' :  current_weather['temp_min'],
        'MaxTemp' : current_weather['temp_max'],
        'WindGustDir' : compass_direction_encoded,
        'WindGustSpeed' : current_weather['Wind_gust_Speed'],
        'Humidity' : current_weather['humidity'],
        'Pressure' : current_weather['pressure'],
        'Temp' : current_weather['current_temp'],
    }
    
    current_df = pd.DataFrame([current_data])
    
    rain_prediction = rain_model.predict(current_df)[0]
    
    x_temp, y_temp = prepare_regression_data(historical_data, 'Temp')
    x_hum, y_hum = prepare_regression_data(historical_data, 'Humidity')
    temp_model = train_regression_model(x_temp, y_temp)
    hum_model = train_regression_model(x_hum, y_hum)
    
    future_temp = predict_future(temp_model, current_weather['temp_min'])
    future_hum = predict_future(hum_model, current_weather['humidity'])
    
    timezome = pytz.timezone('Asia/Karachi')
    now = datetime.now(timezome)
    next_hour = now + timedelta(hours=1)
    next_hour = next_hour.replace(minute=0, second=0, microsecond=0)
    
    print(f"Current weather in {current_weather['city']}, {current_weather['country']}:")
    print(f"Temperature: {current_weather['current_temp']}°C (Feels like {current_weather['feels_like']}°C)")
    print(f"Min Temperature: {current_weather['temp_min']}°C")
    print(f"Max Temperature: {current_weather['temp_max']}°C")
    print(f"Humidity: {current_weather['humidity']}%")
    print(f"Pressure: {current_weather['pressure']} hPa")
    print(f"Wind: {current_weather['Wind_gust_Speed']} m/s, Direction: {compass_direction}")
    print(f"Description: {current_weather['description']}")
    print(f"Rain Tomorrow Prediction: {'Yes' if rain_prediction == 1 else 'No'}")
    print("\n5-Hour Forecast:")
    for i in range(5):
        forecast_time = next_hour + timedelta(hours=i)
        print(f"{forecast_time.strftime('%Y-%m-%d %H:%M')} - Temp: {round(future_temp[i])}°C, Humidity: {round(future_hum[i])}%")
        
weather_view()

KeyError: 'name'